In [1]:
import nibabel as nib
import numpy as np
import random
import os
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy, order_volumes
"""
package to store the intermediate result
"""
import pickle
"""
package to work out RMSE and SSIM
"""
from utils import calc_RMSE, calc_ssim, load_nii_image, filter_mask, calc_percent


2022-08-01 13:12:36.020670: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [3]:
# s03 data repo
s03_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/'
s03_NDI_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/corrected_ndi.nii'
s03_ODI_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/corrected_odi.nii'
s03_FWF_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/corrected_fwf.nii'

In [4]:
filter_mask(s03_path, s03_FWF_path)

original mask has: 88241 of nonzero voxels
filtered mask has: 86494 of nonzero voxels


In [5]:
s03_mask_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/filtered_mask.nii'

In [6]:
s03_mask = load_nii_image(s03_mask_path)

In [7]:
# load the truth data for subject 3
s03_NDI_img = nib.load(s03_NDI_path)
s03_ODI_img = nib.load(s03_ODI_path)
s03_FWF_img = nib.load(s03_FWF_path)
s03_NDI_affine = s03_NDI_img.affine
s03_ODI_affine = s03_ODI_img.affine
s03_FWF_affine = s03_FWF_img.affine
s03_NDI_img_data = s03_NDI_img.get_fdata()
s03_ODI_img_data = s03_ODI_img.get_fdata()
s03_FWF_img_data = s03_FWF_img.get_fdata()

----------
Auxilary functions for AMICO

In [ ]:
# re-fit to ensure the header is preserved
def writefile(path, file, combine, savename):
    with open(os.path.join(path, savename), 'w') as fout:
        read_path = os.path.join(path, file)
        read_file = open(read_path, 'r')
        lines = read_file.readlines()
        for line in lines:
            temp = line.split()
            temp = [e for e, b in zip(temp, combine) if b == 1]
            fout.write(' '.join(e for e in temp))
            fout.write('\n')
    fout.close()

def writediffusion(path, file, combine, savename):
    # the header now is presevred
    img = nib.load(os.path.join(path, file))
    affine = img.affine
    data = img.get_fdata()
    data = data[..., combine==1]
    img = nib.Nifti1Image(data, affine)
    nib.save(img, os.path.join(path, savename))

def run_amico(study_path, subject_path, diffusion, bvals, bvecs, suffix):
    # run amico
    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(os.path.join(study_path, subject_path, bvals), os.path.join(study_path, subject_path, bvecs))
    ae = amico.Evaluation(study_path, subject_path)
    # load the data
    ae.load_data(dwi_filename = diffusion, scheme_filename = bvals+'.scheme', mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels(regenerate=True)
    ae.load_kernels()
    # model fit.
    ae.fit()
    ae.save_results(path_suffix=suffix)

--------------
AMICO over s03 motion data:

In [ ]:
study_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/'

subject_path  = 's04_motion'

1. full volume of s03 motion data

In [ ]:
combine = np.ones(96, dtype=np.int8)
writefile(os.path.join(study_path, subject_path), 'bvals', combine, savename='full_bvals')
writefile(os.path.join(study_path, subject_path), 'bvecs', combine, savename='full_bvecs')
writediffusion(os.path.join(study_path, subject_path), 'diffusion.nii', combine, savename='full_diffusion')

In [ ]:
# run amico
run_amico(study_path, subject_path, 'full_diffusion.nii', 'full_bvals', 'full_bvecs', 'full_volume')

In [ ]:
# get the saved results
icvf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/AMICO/NODDI_full_volume/FIT_ICVF.nii.gz'
isovf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/AMICO/NODDI_full_volume/FIT_ISOVF.nii.gz'
od_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/AMICO/NODDI_full_volume/FIT_OD.nii.gz'

icvf_img = nib.load(icvf_path)
icvf = icvf_img.get_fdata()
isovf_img = nib.load(isovf_path)
isovf = isovf_img.get_fdata()
od_img = nib.load(od_path)
od = od_img.get_fdata()

In [ ]:
# analyze the saved results: rmse and ssim
ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
print(ndi_rmse, odi_rmse, fwf_rmse)
rmse = (ndi_rmse, odi_rmse, fwf_rmse)

ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
odi_ssim = calc_ssim(od, s03_ODI_img_data)
fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
print(ndi_ssim, odi_ssim, fwf_ssim)
ssim = (ndi_ssim, odi_ssim, fwf_ssim)

ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
print(ndi_percent, odi_percent, fwf_percent)
percent = (ndi_percent, odi_percent, fwf_percent)

In [ ]:
path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO'

with open(os.path.join(path, 's04_96_rmse.pickle'), 'wb') as f:
    pickle.dump(rmse, f)
with open(os.path.join(path, 's04_96_ssim.pickle'), 'wb') as f:
    pickle.dump(ssim, f)
with open(os.path.join(path, 's04_96_percent.pickle'), 'wb') as f:
    pickle.dump(percent, f)


In [ ]:
# save the difference map
ndi_diff = s03_NDI_img_data - icvf
odi_diff = s03_ODI_img_data - od
fwf_diff = s03_FWF_img_data - isovf

ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine)
fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine)

nib.save(ndi_diff_img, '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/s04_96_ndi_diff.nii')
nib.save(odi_diff_img, '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/s04_96_odi_diff.nii')
nib.save(fwf_diff_img, '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/s04_96_fwf_diff.nii')

2. Progressively add motion-corrupted data into the training sample for AMICO

In [ ]:
# use the stringnet threshold to filter off data
t0, t1, r0, r1, out, num = 3.0, 2.0, 3.0, 2.0, 0.05, 96

filter_cmd = '--path ' + os.path.join(study_path, subject_path) + ' --t0 ' + str(t0) + ' --t1 ' + str(t1) + ' --r0 ' + str(r0) + ' --r1 ' + str(r1) + ' --out ' + str(out) + ' --num ' + str(num)
filter_args = filter_parser().parse_args(filter_cmd.split())
sorted_index = order_volumes(filter_args)
sorted_index

In [ ]:
movefile = open('/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/move_t0-3.0_t1-2.0_r0-3.0_r1-2.0_out-0.05.txt', 'r')
combine = np.array([int(num) for num in movefile.readline().split(' ')[:-1]])

In [ ]:
selected_indices = np.where(combine==1)[0]
selected_indices

In [ ]:
set(sorted_index[:len(selected_indices)]).difference(set(selected_indices))

In [ ]:
# get the indices of unselected volumes
unselected_indices = np.where(combine==0)[0]
unselected_indices

In [ ]:
s3_bvals = '0 3000 1000 2000 2000 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 3000 2000 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 3000 2000 1000 1000 2000 3000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 2000 1000 3000 0 1000 3000 2000 3000 1000 2000 1000 2000 3000 3000 1000 2000 1000 2000 3000 0 1000 3000 2000 1000 2000 2000 1000 3000 3000 1000 2000 3000 3000 1000'
s3_bvals = s3_bvals.split()
s3_bvals[26]
# I have checked that the first 8 volume in the sorted index has at least three different b vals.

In [ ]:
for i in range(8,len(sorted_index),3):
    combine = np.zeros(len(sorted_index), dtype=np.int8)
    for index in sorted_index[:i]:
        combine[index] = 1
    selected_vols = str(i)
    if i == 8:
        selected_vols = '08'
    # name of the files
    suffix = selected_vols+'_volume'
    bvals = selected_vols+'_bvals'
    bvecs = selected_vols+'_bvecs'
    diffusion = selected_vols+'_diffusion'
    # write files
    writefile(os.path.join(study_path, subject_path), 'bvals', combine, bvals)
    writefile(os.path.join(study_path, subject_path), 'bvecs', combine, bvecs)
    writediffusion(os.path.join(study_path, subject_path), 'diffusion.nii', combine, diffusion)
    # run amico
    run_amico(study_path, subject_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)
    # analyze
    icvf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion/AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    rsme = (ndi_rmse, odi_rmse, fwf_rmse)

    
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    ssim = (ndi_ssim, odi_ssim, fwf_ssim)

    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    percent = (ndi_percent, odi_percent, fwf_percent)

    path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/'

    with open(os.path.join(path, 's04_'+selected_vols+'_rmse.pickle'), 'wb') as f:
        pickle.dump(rsme, f)
    with open(os.path.join(path, 's04_'+selected_vols+'_ssim.pickle'), 'wb') as f:
        pickle.dump(ssim, f)
    with open(os.path.join(path, 's04_'+selected_vols+'_percent.pickle'), 'wb') as f:
        pickle.dump(percent, f)
    
    # save the differen maps
    ndi_diff = s03_NDI_img_data - icvf
    odi_diff = s03_ODI_img_data - od
    fwf_diff = s03_FWF_img_data - isovf

    ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
    odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine)
    fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine)

    nib.save(ndi_diff_img, '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/s04_'+selected_vols+'_ndi_diff.nii')
    nib.save(odi_diff_img, '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/s04_'+selected_vols+'_odi_diff.nii')
    nib.save(fwf_diff_img, '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_AMICO/s04_'+selected_vols+'_fwf_diff.nii')

---------
run with CNN</br>
1. motion free

In [8]:
"""
packages that trains network
"""
from Training import train_network
from utils.model import parser as model_parser
"""
packages that test network
"""
from Testing import test_model

In [9]:
# hyperparameters for the networks
layers = 4 # the number of hidden layers; this is the optimal number of layer
lr = 0.0001 # the learning rate
patch_size = 3 # the size of patches for 2D and 3D CNN
batch = 256 # the batch size
epoch = 100 # the number of epoches for training

# we firstly study subject s02, use s01 as a seperate training dataset
study_subject = 's03_motion' # the subject we want to study
sep_train_subject = 's01_still' # this is separate training candidate
model = 'conv3d'
ltype = 'A'

In [10]:
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)

Generate data

In [ ]:
"""
Generate the base dataset for s03_motion first for all NODDI parameters.
"""
# cmd = "--base --label_type A --subjects s03_motion"
# args = data_parser().parse_args(cmd.split())
# generate_data(args)

In [ ]:
# cnn3ddata_cmd = '--subjects s03_motion' + ' --label_type ' + ltype + ' --' + model
# cnn3ddata_args  = data_parser().parse_args(cnn3ddata_cmd.split())
# generate_data(cnn3ddata_args)

In [11]:
s03_motion_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_motion'

In [12]:
# use the stringnet threshold to filter off data
t0, t1, r0, r1, out, num = 3.0, 2.0, 3.0, 2.0, 0.05, 96

filter_cmd = '--path ' + s03_motion_path + ' --t0 ' + str(t0) + ' --t1 ' + str(t1) + ' --r0 ' + str(r0) + ' --r1 ' + str(r1) + ' --out ' + str(out) + ' --num ' + str(num)
filter_args = filter_parser().parse_args(filter_cmd.split())
sorted_index = order_volumes(filter_args)
sorted_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 60, 74, 65, 59, 32, 55, 75,
       79, 70, 63, 52, 35, 62, 78, 56, 54, 73, 30, 10, 72, 68, 29, 31, 76,
       87, 95, 28, 34, 51, 66, 71, 91, 53, 45, 64, 50, 15, 11, 77, 33, 19,
       18, 23, 61, 92, 17, 85, 67, 58, 27, 48, 88, 22, 20, 47, 26, 83, 42,
       86, 41, 57, 69, 16, 80, 49, 84, 93, 13, 43, 90, 12, 46, 40, 38, 25,
       94, 14, 82, 21, 24, 81, 36, 89, 44, 39, 37])

Training Starts here:

In [ ]:
for i in range(8,len(sorted_index),3):
    combine = np.zeros(len(sorted_index), dtype=np.int8)
    for index in sorted_index[:i]:
        combine[index] = 1
    selected_vols = str(i)

    movefile = ' '.join(map(str, combine))

    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    rmse, ssim, percent = test_model(cnntest_args)

    # save the result
    path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_free_CNN/'

    with open(os.path.join(path, 's03_'+selected_vols+'_rmse.pickle'), 'wb') as f:
        pickle.dump(rmse, f)
    with open(os.path.join(path, 's03_'+selected_vols+'_ssim.pickle'), 'wb') as f:
        pickle.dump(ssim, f)
    with open(os.path.join(path, 's03_'+selected_vols+'_percent.pickle'), 'wb') as f:
        pickle.dump(percent, f)
    
    # save the difference maps
    ndi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-'+selected_vols+'-conv3d-patch_3-base_1-layer_4-label_NDIsynthetic.nii'
    odi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-'+selected_vols+'-conv3d-patch_3-base_1-layer_4-label_ODIsynthetic.nii'
    fwf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-'+selected_vols+'-conv3d-patch_3-base_1-layer_4-label_FWFsynthetic.nii'

    ndi_img = nib.load(ndi_path).get_fdata()
    odi_img = nib.load(odi_path).get_fdata()
    fwf_img = nib.load(fwf_path).get_fdata()

    ndi_diff = s03_NDI_img_data - ndi_img
    odi_diff = s03_ODI_img_data - odi_img
    fwf_diff = s03_FWF_img_data - fwf_img

    ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
    odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine) 
    fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine) 

    nib.save(ndi_diff_img, os.path.join(path, 's04_'+selected_vols+'_ndi_diff.nii'))
    nib.save(odi_diff_img, os.path.join(path, 's04_'+selected_vols+'_odi_diff.nii'))
    nib.save(fwf_diff_img, os.path.join(path, 's04_'+selected_vols+'_fwf_diff.nii'))

In [ ]:
# Train CNN by using all volumes
combine = np.ones(len(sorted_index), dtype=np.int8)
movefile = ' '.join(map(str, combine))

cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
            + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
plot_loss(cnntrain_cmd)
# Testing
cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
cnntest_args = model_parser().parse_args(cnntest_cmd.split())
rmse, ssim, percent = test_model(cnntest_args)

# save the result
path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_free_CNN/'

with open(os.path.join(path, 's03_96_rmse.pickle'), 'wb') as f:
    pickle.dump(rmse, f)
with open(os.path.join(path, 's03_96_ssim.pickle'), 'wb') as f:
    pickle.dump(ssim, f)
with open(os.path.join(path, 's03_96_percent.pickle'), 'wb') as f:
    pickle.dump(percent, f)

ndi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-96-conv3d-patch_3-base_1-layer_4-label_NDIsynthetic.nii'
odi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-96-conv3d-patch_3-base_1-layer_4-label_ODIsynthetic.nii'
fwf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-96-conv3d-patch_3-base_1-layer_4-label_FWFsynthetic.nii'

ndi_img = nib.load(ndi_path).get_fdata()
odi_img = nib.load(odi_path).get_fdata()
fwf_img = nib.load(fwf_path).get_fdata()

ndi_diff = s03_NDI_img_data - ndi_img
odi_diff = s03_ODI_img_data - odi_img
fwf_diff = s03_FWF_img_data - fwf_img

ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine) 
fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine) 

nib.save(ndi_diff_img, os.path.join(path, 's04_96_ndi_diff.nii'))
nib.save(odi_diff_img, os.path.join(path, 's04_96_odi_diff.nii'))
nib.save(fwf_diff_img, os.path.join(path, 's04_96_fwf_diff.nii'))

2. motion

In [13]:
for i in range(8,len(sorted_index),3):
    combine = np.zeros(len(sorted_index), dtype=np.int8)
    for index in sorted_index[:i]:
        combine[index] = 1
    selected_vols = str(i)

    movefile = ' '.join(map(str, combine))

    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    rmse, ssim, percent = test_model(cnntest_args)

    # save the result
    path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_CNN/'

    with open(os.path.join(path, 's04_'+selected_vols+'_rmse.pickle'), 'wb') as f:
        pickle.dump(rmse, f)
    with open(os.path.join(path, 's04_'+selected_vols+'_ssim.pickle'), 'wb') as f:
        pickle.dump(ssim, f)
    with open(os.path.join(path, 's04_'+selected_vols+'_percent.pickle'), 'wb') as f:
        pickle.dump(percent, f)
    
    # save the difference maps
    ndi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-'+selected_vols+'-conv3d-patch_3-base_1-layer_4-label_NDIsynthetic.nii'
    odi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-'+selected_vols+'-conv3d-patch_3-base_1-layer_4-label_ODIsynthetic.nii'
    fwf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-'+selected_vols+'-conv3d-patch_3-base_1-layer_4-label_FWFsynthetic.nii'

    ndi_img = nib.load(ndi_path).get_fdata()
    odi_img = nib.load(odi_path).get_fdata()
    fwf_img = nib.load(fwf_path).get_fdata()

    ndi_diff = s03_NDI_img_data - ndi_img
    odi_diff = s03_ODI_img_data - odi_img
    fwf_diff = s03_FWF_img_data - fwf_img

    ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
    odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine) 
    fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine) 

    nib.save(ndi_diff_img, os.path.join(path, 's04_'+selected_vols+'_ndi_diff.nii'))
    nib.save(odi_diff_img, os.path.join(path, 's04_'+selected_vols+'_odi_diff.nii'))
    nib.save(fwf_diff_img, os.path.join(path, 's04_'+selected_vols+'_fwf_diff.nii'))

8
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 3, 3, 8)]      0         
_________________________________________________________________
conv3d (Conv3D)              (None, 1, 1, 1, 150)      32550     
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 1, 1, 1, 3)     

2022-08-01 13:13:53.105914: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-01 13:13:53.146714: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 13:13:53.146806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.777GHz coreCount: 28 deviceMemorySize: 11.76GiB deviceMemoryBandwidth: 335.32GiB/s
2022-08-01 13:13:53.146828: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-08-01 13:13:53.164213: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-01 13:13:53.186271: I tensorflow/stream_executor/pl

The remained data has shape: (81634, 3, 3, 3, 8)
(81634, 3, 3, 3, 8)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 3)
(81634, 1, 1, 1, 3)
Training start ...


2022-08-01 13:13:56.573475: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-08-01 13:13:56.573516: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 1 GPUs
2022-08-01 13:13:56.577683: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcupti.so.10.1
2022-08-01 13:13:56.580228: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


...

2022-08-01 13:52:51.817640: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-08-01 13:52:51.842594: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-08-01 13:52:51.842900: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-08-01 13:52:51.858015: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_08_01_13_52_51
2022-08-01 13:52:51.858587: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_08_01_13_52_51/vw.trace.json.gz
2022-08-01 13:52:51.858734: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-08-01 13:52:51.861271: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_motion
Model: "functional_173"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_58 (InputLayer)        [(None, 84, 84, 50, 92)]  0         
_________________________________________________________________
conv3d_285 (Conv3D)          (None, 82, 82, 48, 150)   372750    
_________________________________________________________________
conv3d_286 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_287 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_288 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_57 (Dropout)         (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_289 (Conv3D)      

2022-08-01 13:54:54.767777: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-08-01 13:54:54.767958: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-08-01 13:54:55.375399: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-08-01 13:54:55.397159: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-08-01 13:54:55.397428: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-08-01 13:54:55.411436: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_08_01_13_54_55
2022-08-01 13:54:55.411990: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_08_01_13_54_55/vw.trace.json.gz
2022-08-01 13:54:55.412134: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-08-01 13:54:55.414524: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_motion
Model: "functional_179"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_60 (InputLayer)        [(None, 84, 84, 50, 95)]  0         
_________________________________________________________________
conv3d_295 (Conv3D)          (None, 82, 82, 48, 150)   384900    
_________________________________________________________________
conv3d_296 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_297 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_298 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_59 (Dropout)         (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_299 (Conv3D)      

In [14]:
# Train CNN by using all volumes
combine = np.ones(len(sorted_index), dtype=np.int8)
movefile = ' '.join(map(str, combine))

cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
            + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
plot_loss(cnntrain_cmd)
# Testing
cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
cnntest_args = model_parser().parse_args(cnntest_cmd.split())
rmse, ssim, percent = test_model(cnntest_args)

# save the result
path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_CNN/'

with open(os.path.join(path, 's04_96_rmse.pickle'), 'wb') as f:
    pickle.dump(rmse, f)
with open(os.path.join(path, 's04_96_ssim.pickle'), 'wb') as f:
    pickle.dump(ssim, f)
with open(os.path.join(path, 's04_96_percent.pickle'), 'wb') as f:
    pickle.dump(percent, f)

ndi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-96-conv3d-patch_3-base_1-layer_4-label_NDIsynthetic.nii'
odi_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-96-conv3d-patch_3-base_1-layer_4-label_ODIsynthetic.nii'
fwf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/nii/s04_motion-96-conv3d-patch_3-base_1-layer_4-label_FWFsynthetic.nii'

ndi_img = nib.load(ndi_path).get_fdata()
odi_img = nib.load(odi_path).get_fdata()
fwf_img = nib.load(fwf_path).get_fdata()

ndi_diff = s03_NDI_img_data - ndi_img
odi_diff = s03_ODI_img_data - odi_img
fwf_diff = s03_FWF_img_data - fwf_img

ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine) 
fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine) 

nib.save(ndi_diff_img, os.path.join(path, 's04_96_ndi_diff.nii'))
nib.save(odi_diff_img, os.path.join(path, 's04_96_odi_diff.nii'))
nib.save(fwf_diff_img, os.path.join(path, 's04_96_fwf_diff.nii'))

96
Model: "functional_181"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_61 (InputLayer)        [(None, 3, 3, 3, 96)]     0         
_________________________________________________________________
conv3d_300 (Conv3D)          (None, 1, 1, 1, 150)      388950    
_________________________________________________________________
conv3d_301 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_302 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_303 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_60 (Dropout)         (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_304 (Conv3D)          (None, 1, 1, 1, 3)  

2022-08-01 13:57:04.010812: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-08-01 13:57:04.011213: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-08-01 13:57:04.627988: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-08-01 13:57:04.650003: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-08-01 13:57:04.650269: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-08-01 13:57:04.678759: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_08_01_13_57_04
2022-08-01 13:57:04.679285: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_08_01_13_57_04/vw.trace.json.gz
2022-08-01 13:57:04.679417: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-08-01 13:57:04.681837: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

Test subject is s03_motion
Model: "functional_185"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_62 (InputLayer)        [(None, 84, 84, 50, 96)]  0         
_________________________________________________________________
conv3d_305 (Conv3D)          (None, 82, 82, 48, 150)   388950    
_________________________________________________________________
conv3d_306 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_307 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_308 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_61 (Dropout)         (None, 82, 82, 48, 150)   0         
_________________________________________________________________
conv3d_309 (Conv3D)      

-----
AMICO on motion-free data

In [ ]:
subject_path = 's04_still_reg'

1. full volume of s03 motion free data

In [ ]:
combine = np.ones(96, dtype=np.int8)
writefile(os.path.join(study_path, subject_path), 'bvals', combine, savename='full_bvals')
writefile(os.path.join(study_path, subject_path), 'bvecs', combine, savename='full_bvecs')
writediffusion(os.path.join(study_path, subject_path), 'diffusion.nii', combine, savename='full_diffusion')

In [ ]:
# run amico
run_amico(study_path, subject_path, 'full_diffusion.nii', 'full_bvals', 'full_bvecs', 'full_volume')

In [ ]:
# get the saved results
icvf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/AMICO/NODDI_full_volume/FIT_ICVF.nii.gz'
isovf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/AMICO/NODDI_full_volume/FIT_ISOVF.nii.gz'
od_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/AMICO/NODDI_full_volume/FIT_OD.nii.gz'

icvf_img = nib.load(icvf_path)
icvf = icvf_img.get_fdata()
isovf_img = nib.load(isovf_path)
isovf = isovf_img.get_fdata()
od_img = nib.load(od_path)
od = od_img.get_fdata()

In [ ]:
# analyze the saved results: rmse and ssim
ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
print(ndi_rmse, odi_rmse, fwf_rmse)
RMSE = (ndi_rmse, odi_rmse, fwf_rmse)

ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
odi_ssim = calc_ssim(od, s03_ODI_img_data)
fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
print(ndi_ssim, odi_ssim, fwf_ssim)
SSIM = (ndi_ssim, odi_ssim, fwf_ssim)

ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
Percent = (ndi_percent, odi_percent, fwf_percent)

In [ ]:
path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_free_AMICO'

with open(os.path.join(path, 's04_96_rmse.pickle'), 'wb') as f:
    pickle.dump(RMSE, f)
with open(os.path.join(path, 's04_motion_free_96_ssim.pickle'), 'wb') as f:
    pickle.dump(SSIM, f)
with open(os.path.join(path, 's04_96_percent.pickle'), 'wb') as f:
    pickle.dump(Percent, f)

In [ ]:
# save the difference maps
ndi_diff = s03_NDI_img_data - icvf
odi_diff = s03_ODI_img_data - od
fwf_diff = s03_FWF_img_data - isovf

ndi_diff_img = nib.Nifti1Image(ndi_diff, s03_NDI_affine)
odi_diff_img = nib.Nifti1Image(odi_diff, s03_ODI_affine)
fwf_diff_img = nib.Nifti1Image(fwf_diff, s03_FWF_affine)

os.path.join(path, 's04_96_ndi_diff.nii')

nib.save(ndi_diff_img, os.path.join(path, 's04_96_ndi_diff.nii'))
nib.save(odi_diff_img, os.path.join(path, 's04_96_odi_diff.nii'))
nib.save(fwf_diff_img, os.path.join(path, 's04_96_fwf_diff.nii'))

2. Progressively add motion-corrupted data into the training sample for AMICO

In [ ]:
# use the stringnet threshold to filter off data
t0, t1, r0, r1, out, num = 3.0, 2.0, 3.0, 2.0, 0.05, 96

filter_cmd = '--path ' + s03_motion_path + ' --t0 ' + str(t0) + ' --t1 ' + str(t1) + ' --r0 ' + str(r0) + ' --r1 ' + str(r1) + ' --out ' + str(out) + ' --num ' + str(num)
filter_args = filter_parser().parse_args(filter_cmd.split())
sorted_index = order_volumes(filter_args)
sorted_index

In [ ]:
for i in range(8,len(sorted_index),3):
    combine = np.zeros(len(sorted_index), dtype=np.int8)
    for index in sorted_index[:i]:
        combine[index] = 1
    selected_vols = str(i)
    # name of the files
    suffix = selected_vols+'_volume'
    bvals = selected_vols+'_bvals'
    bvecs = selected_vols+'_bvecs'
    diffusion = selected_vols+'_diffusion'
    # write files
    writefile(os.path.join(study_path, subject_path), 'bvals', combine, bvals)
    writefile(os.path.join(study_path, subject_path), 'bvecs', combine, bvecs)
    writediffusion(os.path.join(study_path, subject_path), 'diffusion.nii', combine, diffusion)
    # run amico
    run_amico(study_path, subject_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)
    # analyze
    icvf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    rsme = (ndi_rmse, odi_rmse, fwf_rmse)

    
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    ssim = (ndi_ssim, odi_ssim, fwf_ssim)

    ndi_percent = calc_percent(icvf, s03_NDI_img_data, s03_mask, threshold=0.1)
    odi_percent = calc_percent(od, s03_ODI_img_data, s03_mask, threshold=0.1)
    fwf_percent = calc_percent(isovf, s03_FWF_img_data, s03_mask, threshold=0.1)
    percent = (ndi_percent, odi_percent, fwf_percent)

    path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp1_results/s4/motion_free_AMICO/'

    with open(os.path.join(path, 's04_'+selected_vols+'_rmse.pickle'), 'wb') as f:
        pickle.dump(rsme, f)
    with open(os.path.join(path, 's04_'+selected_vols+'_ssim.pickle'), 'wb') as f:
        pickle.dump(ssim, f)
    with open(os.path.join(path, 's04_'+selected_vols+'_percent.pickle'), 'wb') as f:
        pickle.dump(percent, f)
    
    # save the difference maps
    nib.save(ndi_diff_img, os.path.join(path, 's04_'+selected_vols+'_ndi_diff.nii'))
    nib.save(odi_diff_img, os.path.join(path, 's04_'+selected_vols+'_odi_diff.nii'))
    nib.save(fwf_diff_img, os.path.join(path, 's04_'+selected_vols+'_fwf_diff.nii'))